# Implementation of manual corrections (in vector and raster form)

# Fixing national data based on a vector file first converted to a raster file

processing.run("native:mergevectorlayers", {'LAYERS':['Path/Fixes1.shp','Path/Fixes2.shp'],'CRS':QgsCoordinateReferenceSystem('EPSG:3067'),'OUTPUT':'FixesAll'})
->field calculator, new numerical field based on a text description of manual correction (correction code)

processing.run("gdal:rasterize", {'INPUT':'Path/FixesAll.gpkg|layername=FixesAll','FIELD':'Correction code','BURN':0,'USE_Z':False,'UNITS':0,'WIDTH':69600,'HEIGHT':118800,'EXTENT':'44000.000000000,740000.000000000,6594000.000000000,7782000.000000000 [EPSG:3067]','NODATA':0,'OPTIONS':'COMPRESS=LZW|BIGTIFF=IF_NEEDED|SPARSE_OK=YES|TILED=YES','DATA_TYPE':5,'INIT':None,'INVERT':False,'EXTRA':'','OUTPUT':'Path/Fixraster.tif'})

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

out = r'Path\Output\Fixed1.tif'

# Read the data with chunking
OG = rioxarray.open_rasterio(r"Path\Input\MaskWithStreamvalues.tif", chunks={'band': 1, 'x': 1024, 'y': 1024})
Fix = rioxarray.open_rasterio(r"Path\Input\Fixraster.tif", chunks={'band': 1, 'x': 1024, 'y': 1024})

OG_data = OG[0]
print('OG read')
Fix_data = Fix[0]
print('Fix read')

# Apply the conditions
#jos korjausaineistossa kommenttina "tämä ojittamattomaksi" -> fix_data=10 ja ojitetut luokat 11 sekä 21 muutetaan ojittamattomaan luokkaan 10 ja 20
#ts kun og_data=11 ja fix_data=10, anna arvoksi 10, muutoin pidä OG eri original arvo
OG_data = da.where((OG_data == 11) & (Fix_data == 10), 10, OG_data)
OG_data = da.where((OG_data == 21) & (Fix_data == 10), 20, OG_data)
#jos aineistossa "puuttuneet ojitukset" -> fix_data=11 ja ojittomattomat luokat 10 sekä 20 muutetaan ojitettuun luokkaan 11 ja 21
OG_data = da.where((OG_data == 10) & (Fix_data == 11), 11, OG_data)
OG_data = da.where((OG_data == 20) & (Fix_data == 11), 21, OG_data)
#jos korjausaineistossa kommenttina "entinen pelto" -> fix_data=22 ja turveluokat 20 sekä 21 muutetaan luokkaan 22
OG_data = da.where((OG_data == 20) & (Fix_data == 22), 22, OG_data)
OG_data = da.where((OG_data == 21) & (Fix_data == 22), 22, OG_data)
#jos korjausaineistossa kommenttina "entinen/nykyinen turvetuotantoalue" -> fix_data=23 ja turveluokat 20 sekä 21 muutetaan luokkaan 23
OG_data = da.where((OG_data == 20) & (Fix_data == 23), 23, OG_data)
OG_data = da.where((OG_data == 21) & (Fix_data == 23), 23, OG_data)

print('conditions set')


# Specify the compression type, sparse_ok, and tiled options
compression_options = { 
"compress": "lzw",
"sparse_ok": True,
"tiled": True
}

# Save the result to a new raster file
result_xarray = xr.DataArray(OG_data, dims=("y", "x"), coords={"y": OG.y, "x": OG.x})
result_xarray.rio.write_crs(OG.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='float32', **compression_options)


OG read
Fix read
conditions set


# Fixing Lapland based on a raster in smaller dimensions

## First change all the inputs to same dimensions

processing.run("gdal:warpreproject", {'INPUT':'Path/SmallerDimensionRaster.tif','SOURCE_CRS':QgsCoordinateReferenceSystem('EPSG:3067'),'TARGET_CRS':QgsCoordinateReferenceSystem('EPSG:3067'),'RESAMPLING':0,'NODATA':None,'TARGET_RESOLUTION':None,'OPTIONS':'COMPRESS=LZW|SPARSE_OK=YES','DATA_TYPE':0,'TARGET_EXTENT':'44000.000000000,740000.000000000,6594000.000000000,7782000.000000000 [EPSG:3067]','TARGET_EXTENT_CRS':QgsCoordinateReferenceSystem('EPSG:3067'),'MULTITHREADING':True,'EXTRA':'','OUTPUT':'Path/SmallerDimensionRaster_WarpedToLargerScale.tif'})

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

out = r'Path\Output\Fixed2.tif'

# Read the data with chunking
chunksize = 1000
chunksize = 'auto'
OG = rioxarray.open_rasterio(r"Path\Output\Fixed1.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})
Lappi = rioxarray.open_rasterio(r"Path\SmallerDimensionRaster_WarpedToLargerScale.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})
TiePeltoTurv = rioxarray.open_rasterio(r"Path\RoadFieldPeatprod.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})

OG_data = OG[0]
print('OG read')
Lappi_data = Lappi[0]
print('Lappi read')
TiePeltoTurv_data = TiePeltoTurv[0]
print('TiePeltoTurv read')


# Apply the conditions
#jos lappidata=1, alueella ei pitäisi olla ojia, mutta jos alueella tietä,peltoa tai turvemaata, arvo muutetaan ojitetuksi=2, muuten alkuperäinen arvo
Lappi_data = da.where((Lappi_data == 1) & (TiePeltoTurv_data == 1), 2, Lappi_data)
#jos ojitettu kivennäismaa lapissa, muuta ojittamattomaksi, muuten alkuperäinen arvo
OG_data = da.where((OG_data == 11) & (Lappi_data == 1), 10, OG_data)
#jos ojitettu turvemaa lapissa, muuta ojittamattomaksi, muuten alkuperäinen arvo
OG_data = da.where((OG_data == 21) & (Lappi_data == 1), 20, OG_data)

print('conditions set')


# Specify the compression type, sparse_ok, and tiled options
compression_options = { 
"compress": "lzw",
"sparse_ok": True,
"tiled": True
}

# Save the result to a new raster file
result_xarray = xr.DataArray(OG_data, dims=("y", "x"), coords={"y": OG.y, "x": OG.x})
result_xarray.rio.write_crs(OG.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='float32', **compression_options)

OG read
Lappi read
TiePeltoTurv read
conditions set


# Creation of a check up raster with only the changed areas

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

out = r'Path\Output\ChangedAreas.tif'

# Read the data with chunking
chunksize = 'auto'
OG = rioxarray.open_rasterio(r"Path\Input\MaskWithStreamvalues.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})
Fix = rioxarray.open_rasterio(r"Path\Output\Fixed2.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})


OG_data = OG[0]
print('OG read')
Fix_data = Fix[0]
print('Fix read')

# Apply the conditions
#jos korjausaineistossa kommenttina "tämä ojittamattomaksi" -> fix_data=10 ja ojitetut luokat 11 sekä 21 muutetaan ojittamattomaan luokkaan 10 ja 20
#ts kun og_data=11 ja fix_data=10, anna arvoksi 10, muutoin pidä OG eri original arvo
OG_data = da.where((OG_data != Fix_data), 1, 0)

print('conditions set')


# Specify the compression type, sparse_ok, and tiled options
compression_options = { 
"compress": "lzw",
"sparse_ok": True,
"tiled": True
}

# Save the result to a new raster file
result_xarray = xr.DataArray(OG_data, dims=("y", "x"), coords={"y": OG.y, "x": OG.x})
result_xarray.rio.write_crs(OG.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='float32', **compression_options)


OG read
Fix read
conditions set


# Additional checkups and corrections

First convert vector to raster
Here agricultural lands are buffered 50m and updated with a newer version
processing.run("gdal:rasterize", {'INPUT':'Path/AgriculturalLandBuff50m.gpkg|layername=AgriculturalLandBuff50m','FIELD':'kohdeluokka','BURN':0,'USE_Z':False,'UNITS':0,'WIDTH':69600,'HEIGHT':118800,'EXTENT':'44000.000000000,740000.000000000,6594000.000000000,7782000.000000000 [EPSG:3067]','NODATA':0,'OPTIONS':'COMPRESS=LZW|SPARSE_OK=YES|TILED=YES','DATA_TYPE':5,'INIT':None,'INVERT':False,'EXTRA':'','OUTPUT':'Path/AgriculturalLandBuff50m.tif'})

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

out = r"Path\Fixed3.tif"

# Read the data with chunking
chunksize = 'auto'
OG = rioxarray.open_rasterio(r"Path\Fixed2.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})
Fix = rioxarray.open_rasterio(r"Path\AgriculturalLandBuff50m.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})


OG_data = OG[0]
print('OG read')
Fix_data = Fix[0]
print('Fix read')

# Apply the conditions
#jos fix datassa arvo (maatalousmaan puskurialueella), muuta ojittamaton ojitetuksi
OG_data = da.where((OG_data == 10) & (Fix_data != 0), 11, OG_data)
OG_data = da.where((OG_data == 20) & (Fix_data != 0), 21, OG_data)

print('conditions set')

# Specify the compression type, sparse_ok, and tiled options
compression_options = { 
"compress": "lzw",
"sparse_ok": True,
"tiled": True
}

# Save the result to a new raster file
result_xarray = xr.DataArray(OG_data, dims=("y", "x"), coords={"y": OG.y, "x": OG.x})
result_xarray.rio.write_crs(OG.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='float32', **compression_options)


OG read
Fix read
conditions set


# If certain classes seem unnecessary, convert them to nan here

e.g. classes 0,1,2,22 where
0 is nan
1 is ditch raster not intersecting background mask (not peat land or mineral soil)
2 is ditch raster not intersecting background mask (not peat land or mineral soil)
22 is an unsatisfactory class. Class coverage was deemed lacking and misleading

In [ ]:
import rioxarray
import xarray as xr
import numpy as np
import dask.array as da

out = r"Path\Fixed4.tif"

# Read the data with chunking
chunksize = 'auto'
OG = rioxarray.open_rasterio(r"Path\Fixed3.tif", chunks={'band': 1, 'x': chunksize, 'y': chunksize})

OG_data = OG[0]
print('OG read')

# Apply the condition
OG_data = da.where(((OG_data == 0) | (OG_data == 1) | (OG_data == 2) | (OG_data == 22)), np.nan, OG_data)

print('conditions set')


# Specify the compression type, sparse_ok, and tiled options
compression_options = { 
"compress": "lzw",
"sparse_ok": True,
"tiled": True
}

# Save the result to a new raster file
result_xarray = xr.DataArray(OG_data, dims=("y", "x"), coords={"y": OG.y, "x": OG.x})
result_xarray.rio.write_crs(OG.rio.crs, inplace=True)
result_xarray.rio.to_raster(out, dtype='float32', **compression_options)

OG read
conditions set


## Sieve isolated pixels (optional)

processing.run("gdal:sieve", {'INPUT':"Path\Fixed3.tif",'THRESHOLD':4,'EIGHT_CONNECTEDNESS':False,'NO_MASK':False,'MASK_LAYER':None,'EXTRA':'','OUTPUT':"Path\Fixed3.tif"})

# Creation of a check up raster with only the changed areas again if necessary
same syntax as before